# Global `plot(...)` with `SmartFigure` Contexts

This notebook demonstrates the global `plot(...)` entry point, how figure contexts
work with `with SmartFigure():` and `with fig:`, and how callbacks automatically
route to the correct figure.


## Setup

We import `plot` and `SmartFigure` from the module. If you are running this inside
the repo, the imports below should work as-is.


In [ ]:
import sympy as sp
from SmartFigure import SmartFigure, plot

x, a = sp.symbols('x a')


## Mode B: `plot(...)` creates a new figure when no context is active

Each call to `plot(...)` below creates and displays a brand-new `SmartFigure`
because we are not inside a `with fig:` context.


In [ ]:
p1 = plot(x, sp.sin(x), id='sin')
p2 = plot(x, sp.cos(x), id='cos')


## Context manager: `with SmartFigure():`

Entering `with SmartFigure():` creates a temporary figure and routes all enclosed
`plot(...)` calls to that figure.


In [ ]:
with SmartFigure():
    plot(x, x + 1, id='line')
    plot(x, sp.sin(x), id='sin')


## Context manager: `with fig:` for an existing figure

The first `with fig:` will auto-display the figure. Subsequent entries just set
the current context without re-displaying.


In [ ]:
fig = SmartFigure()

with fig:  # first entry: auto-displays
    plot(x, x**2, id='parabola')

with fig:  # subsequent entry: no re-display
    plot(x, x**3, id='cubic')


## Recovering the figure from a plot handle

Each plot handle returns its owning `SmartFigure` with `.figure()` so you can
re-enter the context later.


In [ ]:
p = plot(x, sp.exp(-x**2), id='gauss')

with p.figure():
    plot(x, sp.sin(x) * sp.exp(-x**2), id='gauss_sin')


## Callbacks run under the owning figure's context

Parameter-change hooks automatically execute with the owning figure set as the
current context. That means calling the global `plot(...)` inside the callback
updates the same figure (rather than creating a new one).

**Tip:** Use stable `id=` values to update existing plots instead of creating
new plots on every slider change.


In [ ]:
fig_cb = SmartFigure()

with fig_cb:
    plot(x, a * sp.sin(x), parameters=[a], id='a_sin')

def on_param_change(change, fig):
    # This uses the global plot(), but will still target fig_cb
    plot(x, a * sp.cos(x), parameters=[a], id='a_cos')

fig_cb.add_param_change_hook(on_param_change, hook_id='update_cos')
